In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(40000)
user_df = pd.read_csv('Users.csv')
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

<ipython-input-5-40a2cd1e39b2>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('Books.csv')


In [6]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)

In [7]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

In [8]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID',
                                                          columns='unique_id_book',
                                                          values='Book-Rating').fillna(0)

In [9]:
users_books_pivot_matrix_df.head()

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,25846,25847,25848,25849,25850,25851,25852,25853,25854,25855
User-ID,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [12]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [13]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[ 7.52138243e-35, -5.65332352e-37, -1.56461690e-33, ...,
        -6.13196227e-35,  0.00000000e+00,  0.00000000e+00],
       [ 9.52807498e-34, -3.63574790e-36, -1.79397750e-32, ...,
        -8.90844074e-34,  0.00000000e+00,  0.00000000e+00],
       [-4.55955579e-34, -6.79257437e-37,  7.26229112e-33, ...,
         1.22710485e-34,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.56003184e-21,  7.03260946e-25, -5.56982137e-21, ...,
        -1.51392854e-22,  0.00000000e+00,  0.00000000e+00],
       [-1.83385211e-34, -3.05426742e-36,  4.90394305e-33, ...,
        -1.66247560e-34,  0.00000000e+00,  0.00000000e+00],
       [ 8.36364907e-34, -4.88934683e-36, -1.44099755e-32, ...,
        -6.35359515e-34,  0.00000000e+00,  0.00000000e+00]])

In [14]:

def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])


In [18]:
k = 50
movie_id =25
top_n = 3
sliced = Vt.T[:, :k] # representative data

similar_books(book_user_rating, 25, top_cosine_similarity(sliced, movie_id, top_n))

Recommendations for The Street Lawyer: 

LONESOME DOVE
Gianna: Aborted... and Lived to Tell About It (Living Books)
Dolores Claiborne


<ipython-input-14-2c90e294a9eb>:5: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
